# Prepare Data

In [1]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
# Cleaning Twitter data provided by Professor Li
import json
with open('/Users/yangtang/Desktop/Project/tweets-170a-winter20-coronavirus.json') as f:
    data = json.load(f)
#print(data[0])

In [3]:
timeline=[['2020-01-22','2020-01-28'],['2020-01-29','2020-02-04'],['2020-02-05','2020-02-11'],
          ['2020-02-12','2020-02-18'],['2020-02-19','2020-02-25'],['2020-02-26','2020-03-03'],
          ['2020-03-04','2020-03-10'],['2020-03-11','2020-03-17'],['2020-03-18','2020-03-24'],
          ['2020-03-25','2020-03-31']]
datasetsname = ['Jan_TwentySecond','Jan_TwentyNinth','Feb_Fifth','Feb_Twelfth','Feb_Nineteenth',
                'Feb_TwentySixth','Mar_Fourth','Mar_Tenth','Mar_Seventeenth','Mar_TwentyFifth']
#datasets.clear()
datasets={}
for i in datasetsname:
    datasets[i]=[]
#print(datasets)

In [4]:
# Seperate by timeline
for tweet in data:
    if tweet['in_reply_to_status']==-1:
# Remove all replying tweets
        for i in range(len(timeline)):
            if timeline[i][0]<=tweet['create_at'][:10]<=timeline[i][1]:
                datasets[datasetsname[i]].append(tweet)
# for i in datasetsname:
#     print(i,len(datasets[i]))

In [5]:
j=0
for i in datasetsname:
    j+=len(datasets[i])
    print(i,len(datasets[i]))
print(j)

Jan_TwentySecond 5911
Jan_TwentyNinth 7143
Feb_Fifth 4090
Feb_Twelfth 3102
Feb_Nineteenth 4802
Feb_TwentySixth 27692
Mar_Fourth 47015
Mar_Tenth 149765
Mar_Seventeenth 114211
Mar_TwentyFifth 57545
421276


In [204]:
#datasets.clear()

In [ ]:
# https://twitter.com/user/status/

In [6]:
def check_if_contaning_links(datasets:dict,timeline):
    l=[]
    for i in datasets[timeline]:
        print(c)
        url="https://twitter.com/user/status/"+str(i["id"])
        #print(url)
        try:
            soup = BeautifulSoup(urlopen(url))
            productDivs = soup.findAll('div', attrs={'class' : 'js-tweet-text-container'})[0]
            for a in productDivs.find_all('a', href=True):
                
                # check if the raw tweet contains a link to other website or to another tweet
                if a['href'].find("http") == 0:
                    #print("Found the URL:", a['href'])
                    soup2 = BeautifulSoup(urlopen(a['href']))
                    
                    string = soup2.title.string
                    #print(string)
                    url2=a['href']
                    
                    # 3 situation:
                    # 1) Include the tweets that is original and have links
                    #print(string.find("on Twitter")==-1)#->valid
                    f=string.find("on Twitter")
                    if f==-1: #->valid
                        #print("1. IS ORIGINAL")
                        i['links_contained'] = url2
                        l.append(i)
                        #print(url2)
                        break

                    # 2) Include the Retweets the news thread with links inside
                    # 3) Include the Retweets that tweeted one have links
                    elif f!=-1:
                        productDivs = soup.findAll('div', attrs={'class' : "self-thread-context"})

                        
                        # If the link is a thread
                        if len(productDivs) != 0:
                            #print("2. IS THREAD-2")
                            productDivs2 = soup2.findAll('div', attrs={'class' : 'js-tweet-text-container'})[0]
                            for b in productDivs2.find_all('a', href=True):
                                if b['href'].find('http')!=-1 and BeautifulSoup(urlopen(b['href'])).title.string.find("on Twitter")==-1:
                                    #print(b['href'])
                                    i['links_contained'] = b['href']
                                    l.append(i)
                            break
                                    
                        # If the link is not a thread
                        else:
                            #print("3. NOT THREAD")
                            if string.find("http")!=-1:
                                substring = string[string.find("http"):]
                                extract_link = substring[:substring.find(" ")]
                                extract_link = extract_link[:extract_link.find("...")]
                                i['links_contained'] = extract_link
                                l.append(i)
                            break
            
        except:
            # The tweet page is not avaliable
            #print("NOT FOUND")
            pass
        #print()
    #for i in l:
    #    print(i['links_contained'])
    #print(len(l))
    return l

In [209]:
datasets['Jan_TwentySecond']=check_if_contaning_links(datasets,'Jan_TwentySecond')
print(len(datasets['Jan_TwentySecond']))
with open('Jan_TwentySecond', 'w') as json_file:
    json.dump(datasets['Jan_TwentySecond'],json_file)
# with open('Jan_TwentySecond') as f:
#     a = json.load(f)

1790


In [ ]:
datasets['Mar_Fourth']=check_if_contaning_links(datasets,'Mar_Fourth')
print(len(datasets['Mar_Fourth']))
with open('Mar_Fourth', 'w') as json_file:
    json.dump(datasets['Mar_Fourth'],json_file)
# 47015

In [ ]:
datasets['Mar_TwentyFifth']=check_if_contaning_links(datasets,'Mar_TwentyFifth')
print(len(datasets['Mar_TwentyFifth']))
with open('Mar_TwentyFifth', 'w') as json_file:
    json.dump(datasets['Mar_TwentyFifth'],json_file)
# 57545

In [83]:
# Jan_TwentySecond 5911
# Jan_TwentyNinth 7143
# Feb_Fifth 4090
# Feb_Twelfth 3102
# Feb_Nineteenth 4802
# Feb_TwentySixth 27692
# Mar_Fourth 47015
# Mar_Tenth 149765
# Mar_Seventeenth 114211
# Mar_TwentyFifth 57545

# Jan_TwentySecond 1790*
# Jan_TwentyNinth 2482*
# Feb_Fifth 1884*
# Feb_Twelfth 1441*
# Feb_Nineteenth 2191*
# Feb_TwentySixth 9369*  
# Mar_Fourth 14776*
# Mar_Tenth 45106*
# Mar_Seventeenth 44647*
# Mar_TwentyFifth 24759*